# Bulk Delete Groups (From Config File)

This notebook deletes Analysis Groups from Moody's Risk Modeler based on group names defined in an Excel configuration file.

**Use Case:** Clean up groups that were created from a specific configuration file (e.g., test data cleanup).

**Process:**
1. Select an Excel configuration file from the `configuration/` folder
2. Read group names from the "Groupings" tab
3. Search for these groups in Moody's
4. Review found groups and any not found
5. Confirm deletion
6. Delete groups with progress tracking
7. View summary of results

**Warning:** This action permanently deletes groups from Moody's and CANNOT be undone!

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
from datetime import datetime
from helpers import ux
from helpers.irp_integration import IRPClient
from helpers.irp_integration.exceptions import IRPAPIError

# Initialize IRP client
irp_client = IRPClient()

ux.header("Bulk Delete Groups (From Config File)")
ux.info("This tool deletes Analysis Groups based on names in an Excel configuration file.")

## 1. Select Configuration File

Place your Excel configuration file in the `configuration/` folder, then select it below.

In [ ]:
ux.subheader("Select Configuration File")

# Get the configuration directory (relative to this notebook)
config_dir = Path("configuration")
ux.info(f"Configuration directory: {config_dir.absolute()}")
print()

# Check if directory exists
if not config_dir.exists():
    ux.warning(f"Configuration directory does not exist: {config_dir}")
    ux.info("Creating directory...")
    config_dir.mkdir(parents=True, exist_ok=True)
    ux.success("Directory created")

# List available Excel files in the directory
excel_files = sorted(list(config_dir.glob("*.xlsx")) + list(config_dir.glob("*.xls")))

if not excel_files:
    ux.error("No Excel files found in configuration directory")
    print()
    ux.info(f"Please place your configuration file in: {config_dir.absolute()}")
    ux.info("The file should contain a 'Groupings' sheet with a 'Group_Name' column.")
    raise FileNotFoundError("No configuration files found in directory")

# Display available files
ux.info(f"Found {len(excel_files)} Excel file(s):")
print()
for i, f in enumerate(excel_files, 1):
    file_size = f.stat().st_size / 1024  # KB
    file_modified = datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
    ux.info(f"  [{i}] {f.name}")
    ux.info(f"      Size: {file_size:.2f} KB | Modified: {file_modified}")
    print()

# Get user selection
selection = ux.text_input(
    f"Select configuration file [1-{len(excel_files)}]",
    default="1"
)

if selection is None:
    ux.warning("Operation cancelled by user")
    raise Exception("User cancelled operation")

try:
    selected_index = int(selection) - 1
    if selected_index < 0 or selected_index >= len(excel_files):
        raise ValueError("Selection out of range")
    config_path = excel_files[selected_index]
except (ValueError, IndexError) as e:
    ux.error(f"Invalid selection: {selection}")
    raise ValueError(f"Invalid file selection. Please enter a number between 1 and {len(excel_files)}")

# Display selected file information
ux.success(f"Selected: {config_path.name}")
file_size = config_path.stat().st_size / 1024
file_modified = config_path.stat().st_mtime

file_info = [
    ["File Name", config_path.name],
    ["File Size", f"{file_size:.2f} KB"],
    ["Last Modified", datetime.fromtimestamp(file_modified).strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(file_info, headers=["Property", "Value"])

## 2. Read Group Names from Configuration

Reading the "Groupings" tab and extracting all group names from the `Group_Name` column.

In [ ]:
ux.subheader("Reading Configuration File")

try:
    # Read the Groupings sheet
    df = pd.read_excel(config_path, sheet_name='Groupings')
    ux.success(f"Successfully read 'Groupings' sheet")
    
except ValueError as e:
    if "Groupings" in str(e):
        ux.error("'Groupings' sheet not found in the configuration file")
        ux.info("The Excel file must contain a sheet named 'Groupings'")
    raise
except Exception as e:
    ux.error(f"Failed to read configuration file: {e}")
    raise

# Check for Group_Name column
if 'Group_Name' not in df.columns:
    ux.error("'Group_Name' column not found in the Groupings sheet")
    ux.info(f"Available columns: {', '.join(df.columns)}")
    raise ValueError("Missing required 'Group_Name' column")

# Extract unique group names
group_names = df['Group_Name'].dropna().unique().tolist()

if not group_names:
    ux.warning("No group names found in the 'Group_Name' column")
    raise ValueError("No group names to process")

ux.success(f"Found {len(group_names)} unique group name(s) in configuration")
print()

# Display group names
ux.subheader("Group Names from Configuration")
group_list_data = [[i, name] for i, name in enumerate(group_names, 1)]
ux.table(group_list_data, headers=["#", "Group Name"])

## 3. Search for Groups in Moody's

Searching for each group by exact name match in the Moody's API.

In [ ]:
ux.subheader("Searching Moody's API")
ux.info(f"Searching for {len(group_names)} group(s)...")
print()

found_groups = []
not_found_groups = []

for i, group_name in enumerate(group_names, 1):
    try:
        # Search for exact group name match
        filter_str = f'analysisName = "{group_name}" AND engineType = "Group"'
        results = irp_client.analysis.search_analyses(filter=filter_str)
        
        if results:
            found_groups.extend(results)
            ux.success(f"  [{i}/{len(group_names)}] Found: {group_name}")
        else:
            not_found_groups.append(group_name)
            ux.warning(f"  [{i}/{len(group_names)}] Not found: {group_name}")
            
    except IRPAPIError as e:
        not_found_groups.append(group_name)
        ux.error(f"  [{i}/{len(group_names)}] Error searching for: {group_name}")
        ux.warning(f"      {str(e)[:80]}")

print()
ux.info(f"Search complete: {len(found_groups)} found, {len(not_found_groups)} not found")

## 4. Review Groups to Delete

Review the groups that were found and will be deleted.

In [ ]:
# Show groups not found (if any)
if not_found_groups:
    ux.subheader(f"Groups Not Found ({len(not_found_groups)})")
    ux.warning("The following groups were not found in Moody's (already deleted or never created):")
    for name in not_found_groups:
        ux.info(f"  - {name}")
    print()

# Show found groups
if not found_groups:
    ux.warning("No groups found in Moody's to delete")
    ux.info("All groups from the configuration are either already deleted or were never created.")
    raise Exception("No groups to delete")

ux.subheader(f"Groups Found ({len(found_groups)})")

# Build DataFrame for display
groups_data = []
for g in found_groups:
    groups_data.append({
        'Analysis ID': g.get('analysisId'),
        'App Analysis ID': g.get('appAnalysisId'),
        'Group Name': g.get('analysisName'),
        'Peril': g.get('perilCode', 'N/A'),
        'Region': g.get('regionCode', 'N/A'),
        'Framework': g.get('analysisFramework', 'N/A'),
        'Run Date': g.get('createDate', 'N/A')
    })

groups_df = pd.DataFrame(groups_data)
ux.dataframe(groups_df, title=f"Groups to Delete ({len(groups_df)} total)")

# Store for deletion step
groups_to_delete = found_groups.copy()

print()
ux.warning(f"The above {len(groups_to_delete)} group(s) will be PERMANENTLY DELETED from Moody's.")

## 5. Confirm Deletion

**This action cannot be undone!** Please review the groups listed above carefully before confirming.

In [ ]:
ux.header("CONFIRMATION REQUIRED")
ux.error(f"You are about to PERMANENTLY DELETE {len(groups_to_delete)} group(s)")
ux.info(f"Configuration file: {config_path.name}")
print()

# First confirmation
if not ux.yes_no("Are you sure you want to delete these groups?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Second confirmation with count
if not ux.yes_no(f"Final confirmation: DELETE {len(groups_to_delete)} group(s)?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Starting deletion...")
print()

# Track results
deleted_count = 0
failed_count = 0
errors = []

ux.subheader("Deletion Progress")

for i, group in enumerate(groups_to_delete, 1):
    group_name = group.get('analysisName', 'Unknown')
    analysis_id = group.get('analysisId')
    
    try:
        # Delete the group
        irp_client.analysis.delete_analysis(analysis_id)
        deleted_count += 1
        ux.success(f"  [{i}/{len(groups_to_delete)}] Deleted: {group_name}")
        
    except Exception as e:
        failed_count += 1
        error_msg = f"{group_name}: {str(e)}"
        errors.append(error_msg)
        ux.error(f"  [{i}/{len(groups_to_delete)}] FAILED: {group_name}")
        ux.warning(f"      Error: {str(e)[:100]}")

print()

## 6. Deletion Summary

In [ ]:
ux.header("Deletion Summary")

# Summary statistics
summary_data = [
    ["Configuration File", config_path.name],
    ["Groups in Config", len(group_names)],
    ["Groups Found in Moody's", len(found_groups)],
    ["Groups Not Found", len(not_found_groups)],
    ["Successfully Deleted", deleted_count],
    ["Failed to Delete", failed_count],
]

ux.table(summary_data, headers=["Metric", "Value"])
print()

# Overall status
if failed_count == 0 and deleted_count > 0:
    ux.success(f"All {deleted_count} group(s) deleted successfully!")
elif deleted_count == 0 and failed_count > 0:
    ux.error(f"All {failed_count} deletion(s) failed")
elif deleted_count > 0 and failed_count > 0:
    ux.warning(f"Partial success: {deleted_count} deleted, {failed_count} failed")

# Show groups not found
if not_found_groups:
    print()
    ux.subheader("Groups Not Found (Skipped)")
    for name in not_found_groups:
        ux.info(f"  - {name}")

# Show errors if any
if errors:
    print()
    ux.subheader("Deletion Errors")
    for error in errors:
        ux.error(f"  - {error}")

print()
ux.info("Note: Deleted groups cannot be recovered. If you need to recreate them,")
ux.info("you will need to re-run the grouping workflow with the configuration file.")

---